In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import torch
import pandas as pd
from datasets import Dataset
import numpy as np
from transformers import pipeline
from transformers_interpret import MultiLabelClassificationExplainer

# Interpreting the model trained on fake data and tested on real data

In [2]:
dataset_filename = "../preprocessing/data/processed_merge.csv"
# dataset_filename = "../linguistic_model/data/synthetic_clan_train_new.csv"
data = pd.read_csv(dataset_filename, encoding='utf8', index_col=False)

In [3]:
model_name = "models/bert_finetuned_reverse_c4"
tokenizer_name = "models/bert_finetuned_reverse_c4"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, return_tensor="pt")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe = pipeline("text-classification", model=model_name, tokenizer=tokenizer_name, device=device)
cls_explainer = MultiLabelClassificationExplainer(model, tokenizer)

In [4]:
model.config.id2label

{0: 'CONTROL', 1: 'BROCA'}

In [5]:
dataset = Dataset.from_pandas(data)
def preprocess_function(examples):
    return tokenizer(examples["preprocessed_text"], padding=True, truncation=True, return_tensors="pt")
tokenized_data = dataset.map(preprocess_function, batched=True).with_format("torch")
tokenized_data_split = tokenized_data.train_test_split(test_size=0.2, seed=42)

Map:   0%|          | 0/57051 [00:00<?, ? examples/s]

### Observations (model trained on fake data, tested on real)
* Model looks  mostly at nouns, adverbs, verbs and prepositions (existence of these would mean that sentence is most likely control.)
* Model does not look at repetition for classifying broca sentences
* Model also looks at the token ', probably a mistake on my side since the control data does contain contractions but the fake data does not  

In [6]:
num = 0
n = 10
for text, label in zip(list(tokenized_data_split["test"]["preprocessed_text"]), np.array(tokenized_data_split["test"]["label"])): 
    if num != n:
        print("True: ", model.config.id2label[label], "Pred: ", pipe(text)[0]["label"])
        print("Text: ", text)
        cls_explainer(text)
        cls_explainer.visualize()
        print("----------------------------------------------------------------------")
        num += 1

True:  CONTROL Pred:  CONTROL
Text:  i usually use a spoon for my jelly.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.95),CONTROL,1.71,[CLS] i usually use a spoon for my jelly . [SEP]
n/a,(0.03),BROCA,-1.77,[CLS] i usually use a spoon for my jelly . [SEP]


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  the one didn't take the umbrella.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.96),CONTROL,2.06,[CLS] the one didn ' t take the umbrella . [SEP]
n/a,(0.03),BROCA,-2.13,[CLS] the one didn ' t take the umbrella . [SEP]


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  get a scoop with that.


...

----------------------------------------------------------------------
True:  BROCA Pred:  BROCA
Text:  and they one one to.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.04),CONTROL,-1.13,[CLS] and they one one to . [SEP]
n/a,(0.97),BROCA,1.17,[CLS] and they one one to . [SEP]


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  you spread that on the other piece of bread.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.96),CONTROL,1.29,[CLS] you spread that on the other piece of bread . [SEP]
n/a,(0.03),BROCA,-1.31,[CLS] you spread that on the other piece of bread . [SEP]


----------------------------------------------------------------------


# Interpreting the model trained on real data and tested on fake data

In [12]:
# dataset_filename = "../preprocessing/data/processed_merge.csv"
dataset_filename = "../linguistic_model/data/synthetic_clan_merge_c4.csv"
data = pd.read_csv(dataset_filename, encoding='utf8', index_col=False).sample(frac=1)

In [13]:
model_name = "models/bert_finetuned"
tokenizer_name = "models/bert_finetuned"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, return_tensor="pt")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe = pipeline("text-classification", model=model_name, tokenizer=tokenizer_name, device=device)
cls_explainer = MultiLabelClassificationExplainer(model, tokenizer)

In [14]:
model.config.id2label

{0: 'CONTROL', 1: 'BROCA'}

In [15]:
dataset = Dataset.from_pandas(data)
def preprocess_function(examples):
    return tokenizer(examples["modified"], padding=True, truncation=True, return_tensors="pt")
tokenized_data = dataset.map(preprocess_function, batched=True).with_format("torch")
tokenized_data_split = tokenized_data.train_test_split(test_size=0.2, seed=42)

Map:   0%|          | 0/63962 [00:00<?, ? examples/s]

### Observations (model trained on real data, tested on fake)

* Model looks at ' again for same point as mentioned above

In [16]:
num = 0
n = 10
for text, label in zip(list(tokenized_data_split["test"]["modified"]), np.array(tokenized_data_split["test"]["label"])): 
    if num != n:
        print("True: ", model.config.id2label[label], "Pred: ", pipe(text)[0]["label"])
        print("Text: ", text)
        cls_explainer(text)
        cls_explainer.visualize()
        print("----------------------------------------------------------------------")
        num += 1

True:  BROCA Pred:  BROCA
Text:  we until it to up mountains.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.23),BROCA,-0.40,[CLS] we until it to up mountains . [SEP]
n/a,(0.75),CONTROL,0.45,[CLS] we until it to up mountains . [SEP]


----------------------------------------------------------------------
True:  BROCA Pred:  BROCA
Text:  zero.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.24),BROCA,-0.87,[CLS] zero . [SEP]
n/a,(0.76),CONTROL,0.90,[CLS] zero . [SEP]


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  there i found several reviews with one or two stars that summed up my feelings well about the lack of character development, poor editing, feeling that it was shot on the universal back lot mgm's is long gone, and overall impression that it was not going to come close to changing my feelings about ld.


...

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.92),BROCA,2.01,[CLS] all it is is just pointless murders . [SEP]
n/a,(0.11),CONTROL,-2.05,[CLS] all it is is just pointless murders . [SEP]


----------------------------------------------------------------------
True:  BROCA Pred:  CONTROL
Text:  all expresses are my.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.58),BROCA,0.99,[CLS] all expresses are my . [SEP]
n/a,(0.43),CONTROL,-0.97,[CLS] all expresses are my . [SEP]


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  at the video store the other day and recognized it from maxim's list of the fifty greatest b movies, and i decided to rent it and check it out.


----------------------------------------------------------------------
